## 2. Train Data with tensorflow

### Setting


In [1]:
pwd

'/Users/woongjae/Desktop/gradu/project'

### Library call

In [2]:
from glob import glob
import yaml
from IPython.display import Image
import os
import numpy as np

In [3]:
ls

dataset_load1.ipynb    graduation.py          train_v1.ipynb
dataset_load2.ipynb    graduation_projuct_ML/ train_v2.ipynb
dataset_v1/            sample_data/           ultralytics/
dataset_v2/            tensorflow_v1.py


In [4]:
cd dataset_v2

/Users/woongjae/Desktop/gradu/project/dataset_v2


In [5]:
ls

README.dataset.txt   data.yaml            train/
README.roboflow.txt  test/                valid/


In [6]:
cat data.yaml

train: ../train/images
val: ../valid/images
test: ../test/images

nc: 22
names: ['Hoodie', 'Jacket', 'Mid-lenght dress', 'Pants', 'Shirt', 'coat', 'dress', 'fabric', 'jacket', 'jean', 'm2m', 'plain', 'shirt', 'short', 'shorts', 'skirt', 'slacks', 'suit', 'sweat', 'tie', 'tracksuit', 'tshirt']

roboflow:
  workspace: huseyin-ahrh5
  project: 01234
  version: 5
  license: CC BY 4.0
  url: https://universe.roboflow.com/huseyin-ahrh5/01234/dataset/5

In [7]:
train_list = glob("/Users/woongjae/Desktop/gradu/project/dataset_v2/train/images/*.jpg")
valid_list = glob("/Users/woongjae/Desktop/gradu/project/dataset_v2/valid/images/*.jpg")
test_list = glob("/Users/woongjae/Desktop/gradu/project/dataset_v2/test/images/*.jpg")

In [8]:
print('train img :',len(train_list))
print('valid img :',len(valid_list))
print('test img :',len(test_list))

train img : 5540
valid img : 717
test img : 720


In [9]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import cv2

In [12]:
with open('/Users/woongjae/Desktop/gradu/project/dataset_v2/train.text','w') as f:
  f.write('\n'.join(train_list) + '\n')
with open('/Users/woongjae/Desktop/gradu/project/dataset_v2/valid.text','w') as f:
  f.write('\n'.join(valid_list) + '\n')
with open('/Users/woongjae/Desktop/gradu/project/dataset_v2/test.text','w') as f:
  f.write('\n'.join(test_list) + '\n')

In [15]:
with open('/Users/woongjae/Desktop/gradu/project/dataset_v2/data.yaml','r') as f:
  data = yaml.load(f, Loader=yaml.FullLoader)

print(data)

{'train': '../train/images', 'val': '../valid/images', 'test': '../test/images', 'nc': 22, 'names': ['Hoodie', 'Jacket', 'Mid-lenght dress', 'Pants', 'Shirt', 'coat', 'dress', 'fabric', 'jacket', 'jean', 'm2m', 'plain', 'shirt', 'short', 'shorts', 'skirt', 'slacks', 'suit', 'sweat', 'tie', 'tracksuit', 'tshirt'], 'roboflow': {'workspace': 'huseyin-ahrh5', 'project': 668, 'version': 5, 'license': 'CC BY 4.0', 'url': 'https://universe.roboflow.com/huseyin-ahrh5/01234/dataset/5'}}


In [16]:
data['train'] = 'Users/woongjae/Desktop/gradu/project/dataset_v2/train.text'
data['valid'] = 'Users/woongjae/Desktop/gradu/project/dataset_v2/valid.text'

with open('/Users/woongjae/Desktop/gradu/project/dataset_v2/data.yaml','w') as f:
  yaml.dump(data,f)
  
print(data)

{'train': 'Users/woongjae/Desktop/gradu/project/dataset_v2/train.text', 'val': '../valid/images', 'test': '../test/images', 'nc': 22, 'names': ['Hoodie', 'Jacket', 'Mid-lenght dress', 'Pants', 'Shirt', 'coat', 'dress', 'fabric', 'jacket', 'jean', 'm2m', 'plain', 'shirt', 'short', 'shorts', 'skirt', 'slacks', 'suit', 'sweat', 'tie', 'tracksuit', 'tshirt'], 'roboflow': {'workspace': 'huseyin-ahrh5', 'project': 668, 'version': 5, 'license': 'CC BY 4.0', 'url': 'https://universe.roboflow.com/huseyin-ahrh5/01234/dataset/5'}, 'valid': 'Users/woongjae/Desktop/gradu/project/dataset_v2/valid.text'}


In [17]:
EPOCHS = 200
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2
DROPOUT = 0.3

In [24]:
def get_label(file_path, class_names):
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2] == class_names

In [25]:
def process_path(file_path, class_names, img_shape=(224, 224)):
    label = tf.strings.split(file_path, os.path.sep)
    label = label[-2] == class_names

    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, img_shape)
    return img, label

In [26]:
def prepare_for_training(ds, batch_size=32, cache=True, shuffle_buffer_size=1000):
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()

    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    ds = ds.repeat()
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return ds

In [27]:
def load_label(label_path):
    class_names = []
    with open(label_path) as f:
        for line in f:
            line = line.strip()
            class_names.append(line)

    return np.array(class_names)

In [28]:
def show_batch(image_batch, label_batch, class_names):
    size = len(image_batch)
    sub_size = int(size ** 0.5) + 1

    plt.figure(figsize=(10, 10), dpi=80)
    for n in range(size):
        plt.subplot(sub_size, sub_size, n+1)
        plt.subplots_adjust(left=0.125, bottom=0.1, right=0.9, top=0.9, wspace=0.2, hspace=0.5)
        plt.title(class_names[label_batch[n]==True][0].title())
        plt.imshow(image_batch[n])
    plt.show()

In [29]:
def load_data(data_path, label_path, batch=32):
    class_names = load_label(label_path)
    data_dir = pathlib.Path(data_path)
    list_ds = tf.data.Dataset.list_files(str(data_dir / '*/*'))

	# 데이터 확인
    # for f in list_ds.take(5):
    #     print(f.numpy())

    labeled_ds = list_ds.map(lambda x: process_path(x, class_names))
    train_ds = prepare_for_training(labeled_ds, batch_size=batch)

    return train_ds

In [30]:
for img, label in train_dataset.take(1):
    show_batch(img, label, class_names)

NameError: name 'train_dataset' is not defined